src: <https://almondine-song-c43.notion.site/Homework-3-ec8534279e2045479353509d5c3c73a1>

Use this resource as a reference: https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-messages

The following may also be helpful:
- https://www.rareskills.io/post/finite-fields
- https://www.rareskills.io/post/elliptic-curve-addition
- https://www.rareskills.io/post/elliptic-curves-finite-fields

Implement ECDSA from scratch. You want to use the secp256k1 curve. See here for a reference library:  
<https://www.rareskills.io/post/generate-ethereum-address-from-private-key-python>

1. pick a private key
2. generate the public key using that private key (not the eth address, the public key)
3. pick a message **m** and hash it to produce **h** (h can be though of as a 256 bit number)
4. sign **m** using your private key and a randomly chosen nonce k. produce (r, s, h, PubKey)
5. verify (r, s, h, PubKey) is valid

You may use a library for point multiplication, but everything else you must do from scratch.

Pay close attention to the distinction between the curve order and the prime number $p$ we compute the modulus of $y^2=x^3+b \pmod p$.

A few useful libraries to be used:

- [ECPy](https://github.com/cslashm/ECPy)
- [pysha3](https://github.com/5afe/pysha3)
- [py_ecc](https://github.com/ethereum/py_ecc)

In [ ]:
from ecpy.curves import Curve
from ecpy.keys import ECPublicKey, ECPrivateKey
from sha3 import keccak_256

sk = 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80
cv = Curve.get_curve('secp256k1')
pv_key = ECPrivateKey(sk, cv); print(f"pv_key: {pv_key}")
pk = pv_key.get_public_key(); print(f"pk: {pk}")
self_calc_pk = sk * cv.generator; print(f"self_calc_pk: {self_calc_pk}")

x_part = pk.W.x.to_bytes(32, byteorder='big'); print(f"x_part: {list(x_part)}")
y_part = pk.W.y.to_bytes(32, byteorder='big'); print(f"y_part: {list(y_part)}")
concat_x_y = x_part + y_part; print(f"concat: {concat_x_y}")
eth_addr = '0x' + keccak_256(concat_x_y).digest()[-20:].hex(); print(f"eth addr: {eth_addr}")

In [ ]:
from ecpy.curves import Curve
from sha3 import keccak_256
import secrets

# https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-messages

class ECDSA:
    cv = Curve.get_curve("secp256k1")
    
    def __init__(self, sk=None):
        if sk == None: sk = secrets.randbelow(self.cv.order)
        self.sk = sk
        self.pk = None

    def public_key(self):
        if self.pk is not None: return self.pk
        self.pk = self.sk * cv.generator
        return self.pk

    def sign(self, message):
        digest = keccak_256(message).digest()
        h = int.from_bytes(digest, byteorder = "big")
        k = secrets.randbelow(cv.order - 1) + 1;
        R = k * self.cv.generator
        r = R.x
        s = (pow(k, -1, self.cv.order) * (h + r * self.sk)) % self.cv.order
        return (r, s)

    @classmethod
    def verify(cls, message, r, s, pk):
        s_inv = pow(s, -1, cls.cv.order)
        digest = keccak_256(message).digest()
        h = int.from_bytes(digest, byteorder = "big")        
        R_calc = (h * s_inv) * cls.cv.generator + (r * s_inv) * pk
        return R_calc.x == r

def main():
    sk = 43765866871480949447442477281291151637608214855806437531566966568646888625943
    message = "Hello World!"
    signer = ECDSA(sk)
    pk = signer.public_key()
    
    sig = signer.sign(message.encode())
    print(f"signature: {sig}")
    print(f"verify: {ECDSA.verify(message.encode(), sig[0], sig[1], signer.public_key())}")

main()

In [ ]:
from ecpy.ecdsa import ECDSA
from ecpy.keys  import ECPrivateKey
from sha3 import keccak_256

def hash(message):
    return keccak_256(message.encode()).digest()

skInt = 43765866871480949447442477281291151637608214855806437531566966568646888625943
message = "Hello World!"

cv = Curve.get_curve("secp256k1")
sk = ECPrivateKey(skInt, cv)
pk = sk.get_public_key()
print(f"pk: {pk}")

signer = ECDSA()
digest = hash(message)
print(f"hsh: {digest.hex()}, len: {len(digest)}")

sig = signer.sign(digest, sk)
print(f"sig: {sig.hex()}, len: {len(sig)}")
signer.verify(digest, sig, pk)